In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random
from IPython.display import HTML
from datetime import datetime
from IPython.display import display
from multiprocessing.dummy import Pool as ThreadPool
import multiprocessing

start = datetime.now()

N=200

L = np.zeros((N, N))
Lnew = np.zeros((N, N))


L[199, :] = 50  
Lnew[199, :] = 50

step = 5 

x = np.arange(0, N * step, step) 
y = np.arange(0, N * step, step)

a = np.zeros((N,N))
a = a + 0.002

r1 = random.randrange(10, 190)
r2 = random.randrange(10, 190)

r3 = random.randrange(10, 190)
r4 = random.randrange(10, 190)

r5 = random.randrange(10, 190)
r6 = random.randrange(10, 190)

a[r1,r2] = 0.006
a[r3,r4] = 0.006
a[r5,r6] = 0.006

Y, X = np.meshgrid(np.arange(N), np.arange(N))

points = [(r1,r2),(r3,r4),(r5,r6)]
radius = 10

for x, y in points:
    mask = (X - x) ** 2 + (Y - y) ** 2 <= radius ** 2
    a[mask] = 0.006


a_max = 0.006

t=0
dt = (step*step)/(4*a_max)

#iteracje = 50

def stop(L, Lnew):
    return np.abs(np.sum(Lnew) - np.sum(L))


def update_row(args):
    i, L, a, dt, step = args
    row = np.zeros(L.shape[1])
    for j in range(1, L.shape[1] - 1):
        row[j] = (1 - (4 * dt * a[i, j]) / (step * step)) * L[i, j] + dt * a[i, j] * (
            (L[i - 1, j] + L[i + 1, j] + L[i, j - 1] + L[i, j + 1]) / (step * step)
        )
    return i, row


def geotermal(L,Lnew,t):

    data =[]
    times = []
    e = 100

    num_workers = 2
    pool = ThreadPool(processes=num_workers)
    
    #for k in range(1,iteracje):
    while e > 10:
    
        for i in range(1, N-1):
            for j in range(1, N-1):
                Lnew[i, j] = (1 - (4*dt*a[i,j]) / (step*step)) * L[i,j] + dt*a[i,j] * ((L[i-1,j]+L[i+1,j]+L[i,j-1]+L[i,j+1]) / (step*step))  
       
        e = stop(L, Lnew)
        Lnew[:, 0] = L[:, 1]    
        Lnew[:, N-1] = L[:, N-2]
        data.append(Lnew.copy())  
        times.append(t)
        L[:] = Lnew  
        t= t+dt
        print(e)
    return [data,times]
    

data, times =geotermal(L,Lnew,t)

print(type(data))         # Powinno być listą
print(len(data))          # Powinno być iteracje-1
print(type(data[0]))      # Powinno być numpy.ndarray
print(data[0].shape)


fig, ax = plt.subplots()
mat = ax.matshow(data[0], cmap = 'hot')

for x, y in points:
    circle = plt.Circle((y, x), radius, color='red', fill=False, linewidth=2)
    ax.add_patch(circle)

time_text = ax.text(10, 10, '', color='white', fontsize=12)

def update(frame):

    mat.set_array(data[frame])
    current_time = round(times[frame]/(3600*24))
    time_text.set_text(f'Dzień: {current_time}')
    return [mat,time_text]


matplotlib.rcParams['animation.embed_limit'] = 50000
ani = animation.FuncAnimation(fig, update, frames=range(0, len(data), 10), interval=100, repeat=False)

display(HTML(ani.to_jshtml()))

print(f"Liczba klatek: {len(data)}")


elapsed_time = datetime.now() - start
print(f"Czas wykonania: {elapsed_time} sek")

825.0000000000018
755.5555555555529
698.4375
650.2797067901247
609.3569155092555
574.2500629608203
543.8655788733831
517.3468142234597
494.0187824297518
473.3457112106789
454.89925697826584
438.33447968939436
423.37159949272973
409.78206814443183
397.37787644842683
386.0032999390605
375.5284905819826
365.84447301508044
356.85921482205595
348.494522321831
340.6835741681789
333.3689503331698
326.50104789836405
320.0368004883894
313.93863733737453
308.1736324814701
302.7128055991343
297.53054444442023
292.60412528136294
287.9133127099012
283.44002413002454
279.16804709264034
275.08280012960677
271.17112949700095
267.4211357176864
263.8220249594742
260.3639812016736
257.038055873978
253.83607224070875
250.75054227670262
247.77459416593774
244.90190886615892
242.12666443796843
239.44348704618096
236.8474077136052
234.3338240494959
231.89846629352542
229.53736711382226
227.2468346803107
225.02342860299905
222.86393838316872
220.76536407383173
218.72489888760174
216.73991352487428
214.8079420

In [4]:
ani.save("Animation.gif")

NameError: name 'ani' is not defined